In [8]:
import requests, pandas as pd, numpy as np
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import numpy as np, folium
import matplotlib.cm as cm, matplotlib.colors as colors

API = "https://data.boston.gov/api/3/action/datastore_search_sql"
DATASETS = {
    "building_permits": "6ddcd912-32a0-43df-9908-63574f8c7e77",
    "moving_truck_permits": "fde6709d-62a7-4523-a8eb-76eac2004f4b",
}

def find_latlon_cols(res_id):
    q = f'SELECT * FROM "{res_id}" LIMIT 1'
    cols = pd.DataFrame(requests.get(API, params={"sql": q}).json()["result"]["records"]).columns
    L = [c for c in cols if "lat" in c.lower()]
    G = [c for c in cols if "lon" in c.lower() or "long" in c.lower()]
    # prefer exact names if present
    lat = next((c for c in cols if c.lower() in {"y_latitude","latitude"}), (L[0] if L else None))
    lon = next((c for c in cols if c.lower() in {"x_longitude","longitude"}), (G[0] if G else None))
    return lat, lon

def fetch_coords(res_id, lat_col, lon_col, limit=1000):
    out = []
    off = 0
    while True:
        q = f'SELECT "{lat_col}" AS lat, "{lon_col}" AS lon FROM "{res_id}" WHERE "{lat_col}" IS NOT NULL AND "{lon_col}" IS NOT NULL LIMIT {limit} OFFSET {off}'
        r = requests.get(API, params={"sql": q}).json()["result"]["records"]
        if not r: break
        df = pd.DataFrame(r)
        df = df.replace("", np.nan).dropna(subset=["lat","lon"])
        out.append(df.astype(float).to_numpy())
        off += limit
    return np.vstack(out) if out else np.empty((0,2))

def make_grid(all_xy, n, pad=1e-3):
    mn = all_xy.min(0) - pad
    mx = all_xy.max(0) + pad
    lat = np.linspace(mn[0], mx[0], n+1)
    lon = np.linspace(mn[1], mx[1], n+1)
    # use square centers as evaluation points
    latc = (lat[:-1] + lat[1:]) / 2
    lonc = (lon[:-1] + lon[1:]) / 2
    LON, LAT = np.meshgrid(lonc, latc)
    grid = np.column_stack([LAT.ravel(), LON.ravel()])
    return grid, n, (lat, lon)

def kde_on_grid(xy, grid):
    if xy.size == 0:
        return np.zeros(len(grid))
    xy = xy[np.isfinite(xy).all(1)]
    if len(xy) == 0:
        return np.zeros(len(grid))
    kde = gaussian_kde(xy.T, bw_method="scott")
    return kde(grid.T)


# 1) gather coordinates per dataset
coords = {}
for name, rid in DATASETS.items():
    latc, lonc = find_latlon_cols(rid)
    xy = fetch_coords(rid, latc, lonc)
    coords[name] = xy
    print(f"{name}: {len(xy)} points, cols=({latc}, {lonc})")

# 2) build grid over combined extent
all_xy = np.vstack([v for v in coords.values() if v.size])
grid, N, _ = make_grid(all_xy, n=10)

# 3) smoothed distribution function per dataset (Gaussian KDE on grid)
features = {}
for name, xy in coords.items():
    features[name] = kde_on_grid(xy, grid)

# 4) assemble feature table
out = pd.DataFrame({
    "row": np.repeat(np.arange(N), N),
    "col": np.tile(np.arange(N), N),
    "lat": grid[:,0],
    "lon": grid[:,1],
    **features
})

# scale only the feature columns (per-dataset KDE values)
feat_cols = list(features.keys())
scaler = StandardScaler()
out[feat_cols] = scaler.fit_transform(out[feat_cols])

print(out.head())       # normalized feature matrix per grid square
# out is ready for clustering: rows = grid cells, columns = per-dataset features


building_permits: 680913 points, cols=(y_latitude, x_longitude)
moving_truck_permits: 213123 points, cols=(lat, long)
   row  col        lat        lon  building_permits  moving_truck_permits
0    0    0  42.237636 -71.175239         -0.491270             -0.339211
1    0    1  42.237636 -71.152915         -0.486120             -0.339203
2    0    2  42.237636 -71.130591         -0.081084             -0.334985
3    0    3  42.237636 -71.108268         -0.491194             -0.339211
4    0    4  42.237636 -71.085944         -0.491270             -0.339211


In [10]:
# fit clusterers (keep as-is)
X = out[feat_cols].to_numpy(dtype=float)
k = 4
out["km"]  = KMeans(n_clusters=k, n_init=10, random_state=0).fit_predict(X)
out["db"]  = DBSCAN(eps=0.5, min_samples=5).fit_predict(X)
out["hac"] = AgglomerativeClustering(n_clusters=k).fit_predict(X)

# grid cell size from lat/lon extent and N
cell_dlat = (out["lat"].max() - out["lat"].min()) / N
cell_dlon = (out["lon"].max() - out["lon"].min()) / N

def layer_for(df, label_col, name):
    fg = folium.FeatureGroup(name=name, show=True)
    labs = df[label_col].to_numpy()
    uniq = sorted(set(labs))
    cmap = cm.get_cmap("tab20", max(len(uniq), 1))
    # map labels -> colors; DBSCAN noise (-1) gray
    lut = {lab: colors.to_hex(cmap(i/ max(len(uniq)-1, 1))) for i, lab in enumerate(uniq) if lab != -1}
    for _, r in df.iterrows():
        lab = r[label_col]
        color = "#9e9e9e" if lab == -1 else lut[lab]
        lat, lon = float(r["lat"]), float(r["lon"])
        bounds = [
            [lat - cell_dlat/2, lon - cell_dlon/2],
            [lat - cell_dlat/2, lon + cell_dlon/2],
            [lat + cell_dlat/2, lon + cell_dlon/2],
            [lat + cell_dlat/2, lon - cell_dlon/2],
        ]
        folium.Polygon(bounds, color=color, weight=0.5, fill=True, fill_opacity=0.55).add_to(fg)
    return fg

# base map
m = folium.Map(location=[out["lat"].mean(), out["lon"].mean()], zoom_start=12, tiles="cartodbpositron")
m.add_child(layer_for(out, "km",  "K-Means"))
m.add_child(layer_for(out, "db",  "DBSCAN (-1=noise)"))
m.add_child(layer_for(out, "hac", "Hierarchical"))
folium.LayerControl(collapsed=False).add_to(m)

# In Jupyter: just put `m` on the last line to display; or save:
# m.save("boston_clusters.html")
m
